In [1]:
import math
import pandas as pd
import numpy as np
import wave
import os
import sys
import winsound
import pydub
import time
from pydub import AudioSegment
from pathlib import Path

C:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# C = 0, C# = 1, etc.
roots = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

# 1 |-> sus 2, 2 |-> minor 3rd, 3 |-> major 3rd, 4 |-> sus 4
thirds = [1,2,3,4]

# 1 |-> flat 5th, 2 |-> perfect 5th, 3 |-> sharp 5th
fifths = [1,2,3]

# 0|-> no 7th, 1 |-> minor 7th, 2 |-> major 7th
sevenths = [0,1,2]

# 0 |-> no 9th, 1 |-> flat 9th, 2 |-> natural 9th, 3 |-> sharp 9th
ninths = [0, 1, 2, 3]

# 0 |-> no 11th, 1 |-> natural 11th, 2 |-> sharp 11th
elevenths = [0, 1, 2]

# 0 |-> no 13th, 1 |-> flat 13th, 2 |-> natural 13th
thirteenths = [0, 1, 2]

We define a chord to be a 7-tuple (i.e. an element of the direct product of the sets above) satisfying some conditions to eliminate redundancy.
All these conditions can be viewed in Create Chord Dataset. Here are two:

1. There cannot be both a minor 3rd and sharp 9, for these are the same note.

2. Similarly, there cannot be both a flat 5, and sharp 11, for these are the same note.

The size of the direct product of these sets is 15,552.

There are 
-648 7-tuples failing to satify condition 1,
-864 7-tuples failing to satisfy condition 2,
-108 7-tuples that fail both 1 and 2 simultaneously.

By basic principles of combinatorics, one can work out that there are 6,948 different chords (in the sense that they have been defined, some musicians might disagree with this definition).


In [3]:
chords = pd.read_csv(
    r'C:\Users\Owner\Documents\Chord Project\Chord-Tool\chord_dataset.txt', 
    sep=' '
)

print(chords)
print(chords.head(10))

      root  3rd  5th  7th  9th  11th  13th                 name
0        0    1    1    0    0     0     0             Csus2 b5
1        0    1    1    0    0     0     1         Csus2 b5 b13
2        0    1    1    0    0     0     2          Csus2 b5 13
3        0    1    1    0    0     1     0          Csus2 b5 11
4        0    1    1    0    0     1     1      Csus2 b5 11 b13
...    ...  ...  ...  ...  ...   ...   ...                  ...
6943    11    4    3    1    1     0     2     Bsus4 #5 7 b9 13
6944    11    4    3    2    0     0     0        Bsus4 #5 maj7
6945    11    4    3    2    0     0     2     Bsus4 #5 maj7 13
6946    11    4    3    2    1     0     0     Bsus4 #5 maj7 b9
6947    11    4    3    2    1     0     2  Bsus4 #5 maj7 b9 13

[6948 rows x 8 columns]
   root  3rd  5th  7th  9th  11th  13th             name
0     0    1    1    0    0     0     0         Csus2 b5
1     0    1    1    0    0     0     1     Csus2 b5 b13
2     0    1    1    0    0     0   

In [4]:
def chord_name(chord, shift=0):
    # Inputs: chord; an array, shift, an integer
    # the chord array has a choice of root note, and the remaining elements
    # are the choice of extensions
    
    # Outputs: a string which is the name of the chord
    
    name = ""
    
    # Next we define a some dictionaries using our chord note conventions:
    root_dict = {
    0: 'C',
    1: 'C#',
    2: 'D',
    3: 'Eb',
    4: 'E',
    5: 'F',
    6: 'F#',
    7: 'G',
    8: 'G#',
    9: 'A',
    10: 'Bb',
    11: 'B'
    }
    
    third_dict = {
        1: 'sus2',
        2: 'm',
        3: '',
        4: 'sus4'
    }
    
    fifth_dict = {
        1: ' b5',
        2: '',
        3: ' #5'
    }
    
    seventh_dict = {
        0: '',
        1: ' 7',
        2: ' maj7'
    }
    
    ninth_dict = {
        0: '',
        1: ' b9',
        2: ' 9',
        3: ' #9'
    }
    
    eleventh_dict = {
        0: '',
        1: ' 11',
        2: ' #11'
    }
    
    thirteenth_dict = {
        0: '',
        1: ' b13',
        2: ' 13'
    }
    
    chord_array_dict = {
        0: root_dict,
        1: third_dict,
        2: fifth_dict,
        3: seventh_dict,
        4: ninth_dict,
        5: eleventh_dict,
        6: thirteenth_dict
    }
    
    for i in range(7):
        if i == 0:
            name += chord_array_dict[i][chord[i]+shift]
        else:
            name += chord_array_dict[i][chord[i]]
        
    return name

In [5]:
# Utility functions used later

def note_dist(a,b):
    """
    Inputs: a (the first note)
    Inputs: b (the second note)
    Outputs the note 'distance' of two notes, not necessarily a metric
    for example, note_dist(0, 10)=2
    the distance will never exceed 6 (the 7 exception is a hack for voice leading)
    note_dist is a symmetric function; note_dist(a,b)=note_dist(b,a)
    """
    if a < 0 or b < 0:
        return 7
    c = ((a % 12)-(b % 12)) % 12
    d = ((b % 12)-(a % 12)) % 12
    return min(c, d)

def sign(a):
    """sign function of a real number
    """
    if int(a) == 0:
        return 1
    else:
        return int(a/abs(a))

def cartesian_coord(*arrays):
    """
    Outputs the cartesian product of arrays
    """
    swapper = [j for j in arrays]
    a = swapper[0]
    b = swapper[1]
    swapper[0]=b
    swapper[1]=a
    swapped = tuple(swapper)
    grid = np.meshgrid(*swapped)        
    coord_list = [entry.ravel() for entry in grid]
    points = np.vstack(coord_list).T
    points[:, [1, 0]] = points[:, [0, 1]]
    return points

In [34]:
# chord |-> the elements of Z/12Z of a chord
# voicing |-> the elements of Z/12Z of a voicing of a chord
def keys_of_chord(chord):
    """
    Outputs the 'keys' of a chord.
    Example 1: Any octave of C is 0
    Example 2: Any octave of E is 4
    """
    chord = np.array(chord)
    print(chord)
    offset = np.array([0, chord[0]+1, chord[0]+5, chord[0]+9, chord[0]+0, chord[0]+4, chord[0]+7])
    print(offset)
    keys = (chord+offset)%12
    del_inds = [i for i in range(3,7) if chord[i] == 0]
    keys=np.delete(keys, del_inds)
    return keys

def keys_of_voicing(voicing):
    """
    Outputs the 'keys' of a voicing
    """
    voicing = np.array(voicing)
    keys = np.unique(voicing%12)
    return keys

# [0, 3, 7, 10, 2, 5, 8]
print(keys_of_chord([0, 1, 2, 1, 2, 1, 1]))
print(keys_of_chord([0, 1, 2, 1, 2, 0, 0]))
print(keys_of_chord([0, 1, 2, 1, 0, 1, 0]))
print(keys_of_chord([0, 1, 2, 1, 0, 0, 1]))
print(keys_of_chord([0, 1, 2, 1, 1, 2, 2]))

[0 1 2 1 2 1 1]
[0 1 5 9 0 4 7]
[ 0  2  7 10  2  5  8]
[0 1 2 1 2 0 0]
[0 1 5 9 0 4 7]
[ 0  2  7 10  2]
[0 1 2 1 0 1 0]
[0 1 5 9 0 4 7]
[ 0  2  7 10  5]
[0 1 2 1 0 0 1]
[0 1 5 9 0 4 7]
[ 0  2  7 10  8]
[0 1 2 1 1 2 2]
[0 1 5 9 0 4 7]
[ 0  2  7 10  1  6  9]


In [7]:
# chord-tuple |-> voicing of chord such that lowest note is root, remaining notes are chosen randomly
def random_initial_root_voicing(chord):
    oct_cond = 0
    
    keys = keys_of_chord(chord=chord)
    
    if keys[0] <= 3:
        oct_cond = 1
    
    l = len(keys)
    g=[1+oct_cond if n == keys[0] else 3+oct_cond if n == keys[2] else np.random.randint(3, 5) for n in keys]
    g = np.concatenate(([0+oct_cond], g))
    keys = np.concatenate(([keys[0]], keys))
    voice = 12*g+keys
    
    return voice

chord1 = [4, 1, 2, 1, 0, 1, 0]
chord2 = [2, 2, 2, 2, 0, 2, 2]
print("Chord name: ", chord_name(chord1))
print("Chord name: ", chord_name(chord2))

print("Keys of chord 1: ", keys_of_chord(chord1))
print("Keys of chord 2: ", keys_of_chord(chord2))

print("Random voicing of chord1: ", random_initial_root_voicing(chord1))
print("Keys of random voicing of chord1: ", keys_of_voicing(random_initial_root_voicing(chord1)))
print("Random voicing of chord2: ", random_initial_root_voicing(chord2))

Chord name:  Esus2 7 11
Chord name:  Dm maj7 #11 13
Keys of chord 1:  [ 4  6 11  2  9]
Keys of chord 2:  [ 2  5  9  1  8 11]
Random voicing of chord1:  [ 4 16 54 47 50 57]
Keys of random voicing of chord1:  [ 2  4  6  9 11]
Random voicing of chord2:  [14 26 41 57 49 44 47]


In [8]:
chord1 = [0, 4, 2, 1, 0, 0, 0]

def play_voicing(
    chord, 
    voice=np.array([]), 
    folder=r"C:\Users\Owner\Documents\Chord Project\Chord-Tool\Rhodes Notes"
    ):
    if not voice.any():
        voice = random_initial_root_voicing(chord)
    clip_folder = Path(folder)
    clips = [clip_folder / f'2RhodesNote-0{i}.wav' 
             if i < 10 else clip_folder / f'2RhodesNote-{i}.wav' 
             for i in voice]
    sounds = [AudioSegment.from_file(clip, format="wav")-20 
              for i, clip in enumerate(clips)]
    s = sounds[0]
    for i in range(len(sounds)-1):
        s=s.overlay(sounds[i+1]-2, position = i*50)
    
    combined = s+8
    
    combined.export(clip_folder / 'temp.wav', format = 'wav')
    
    winsound.PlaySound(str(clip_folder / 'temp.wav'), winsound.SND_FILENAME)
    
play_voicing(chord1, voice=random_initial_root_voicing(chord1))

In [9]:
# Replace play_voicing with this
# Returns a pydub AudioSegment of a voicing
def audio_from_voicing(chord,
                       voice=np.array([]),
                       ntuple = 4,
                       duration = 2000, 
                       tempo = 120, 
                       folder='E:\Academics\Jupyter Notebooks\data\Rhodes Notes'
                      ):
    if not voice.any():
        voice = random_initial_root_voicing(chord)
    clip_folder = Path(folder)
    clips = [clip_folder / f'2RhodesNote-0{i}.wav' 
             if i < 10 else clip_folder / f'2RhodesNote-{i}.wav' 
             for i in voice]
    sounds = [AudioSegment.from_file(clip, format="wav")-20 
              for i, clip in enumerate(clips)]
    
    #merges sound clips into one sound
    s = sounds[0]
    for i in range(len(sounds)-1):
        s = s.overlay(sounds[i+1]-2, position = i*50)
    s = s+5
    
    #converts tempo to ms/b
    #needs exception handling or something
    tempo = 60000/tempo
    if ntuple*tempo-duration <= 0.0:
        s = s[:duration]
    else:
        silence = AudioSegment.silent(duration=(ntuple*tempo-duration))
        s = s[:duration]+silence
    return s

In [10]:
def audio_from_chord_progression(chord_prog, voice_prog, ntuples, durations, tempo):
    """
    Inputs: 
    chord_prog, an array of names of chords
    voice_prog, an array of voicings for each chord
    ntuples, an array of tuplet sizes for each chord
    durations, an array of durations for each chord
    tempo, the desired tempo for playback
    
    Outputs:
    array of audio segments, summing the array gives audio for the chord progression"""
    chord_audio = [audio_from_voicing(chord_prog[i], voice_prog[i], ntuples[i], durations[i], tempo) 
                   for i in range(len(chord_prog))]
    s = chord_audio[0]
    for i in range(len(chord_audio)-1):
        s = s + chord_audio[i+1]
    return s

In [11]:
for i, row in chords.sample(frac=1).iterrows():
#for index, row in chords.iterrows():
    print(chord_name(row[:7]))
    chord = row[:7]
    voice = random_initial_root_voicing(chord)
    # comment the line below out to disable rootless voicings
    voice = voice[1:]
    play_voicing(chord, voice=voice)

Eb #9 b13
E 7 9 11
G# #5 maj7 b9 13
G b5
B 9 11 13
Bm maj7 9
Bsus4 7 #11 b13


KeyboardInterrupt: 

In [11]:
# Function that returns index of input chord name
def chord_finder(chord_name):
    return chords.loc[chords['name']==chord_name, chords.columns != 'name'].values.flatten()

In [12]:
# if the root is below 4, raise octave up 1
def voice_correction(voicing):
    """
    Ensures voicing is within range of audio clips
    """
    if voicing[0] < 4:
        voicing[0]+=12
        voicing[1]+=12
    for i in range(len(voicing)):
        if voicing[i] < 27 and i > 1:
            voicing[i]+=12
        elif voicing[i] > 58:
            voicing[i]-=12
        if voicing[i] // 12 > 4:
            voicing[i] -= 12
        if voicing[i] // 12 < 2 and i > 2:
            voicing[i] += 12
    copy = sorted(voicing)
    if (copy[len(copy)-1] - copy[len(copy)-2]) > 8:
        voicing[len(voicing)-1]-=12
    return voicing

In [30]:
def dissonance(a, b):
    """
    Inputs: a, b keys of a voicing, or notes of a chord

    Outputs: dissonance_value, 
    a subjective value for dissonance between a, b
    """
    dist = note_dist(a, b)
    dissonance_value = 0
    if dist == 1 or dist == 6:
        dissonance_value = 2
    elif dist == 2:
        dissonance_value = 1

    return dissonance_value

def chord_dissonance(chord):
    """Inputs: chord, a chord with which the subjective dissonance will
    be measured.

    Outputs: chord_dissonance_value, 
    a subjective value for the dissonance of an entire chord
    """

    
    notes = keys_of_chord(chord)
    n = len(notes)

    chord_dissonance_value = 0
    for i in range(n):
        for j in range(i, n):
            if j > i:
                diss = dissonance(notes[i], notes[j])
                chord_dissonance_value += diss

    return chord_dissonance_value

a = dissonance(0, 4)
b = dissonance(0, 7)
c = dissonance(4, 7)
print(a+b+c)
print(chord_dissonance(chord_finder("Csus2 b5 maj7 b9 11 13")))

0
15


In [31]:
chords["dissonance"] = 0

In [ ]:
for i in range(579):
    chord = chord_finder(chords["name"].iloc[i])
    chord = chord[:-1]
    chords["dissonance"].iloc[i] = chord_dissonance(chord)

print(chords)

In [ ]:
for i in range(579, 6949):
    chords["dissonance"].iloc[i] = chords["dissonance"].iloc[i % 579]

In [42]:
column = chords["dissonance"]
print(column.max())

19


In [41]:
chords.to_csv(
    r'C:\Users\Owner\Documents\Chord Project\Chord-Tool\chords dataset2.txt', 
    header='infer', 
    index=None, 
    sep=' ', 
    mode='a'
)

In [ ]:
#chord_progression=['Bm 13', 'C#m 13', 'Dm 13', 'C#m 13']
#chord_progression=['Bm', 'Gm', 'Bbm', 'C#m 13']
chord_progression=['G#sus4', 'Am 7', 'C#m 13', 'Cm 13', 'F#sus4', 'G# 7', 'C 7', 'B 7']
for i in range(4):
    for i, chord in enumerate(chord_progression):
        chord1 = chord_finder(chord)
        voicing = random_initial_root_voicing(chord1)
        voicing = voice_correction(voicing)
        play_voicing(chord_finder(chord), voice = voicing)

KeyboardInterrupt: 

$\large
\textbf{Conditional Chord Voicing:} \newline \normalsize$
For a chord $c$ with voicing $v$, let $N(c, v)$ be the tuple of numerical notes on a keyboard to play that chord-voicing pair. For example, if $c=(0, 2, 2, 0, 0, 2, 0), v=(3, 4, 3, 0, 0, 4, 0)$, i.e. a Cmaj #11 chord containing notes (C, G, E, F#), then $$N(c, v)=(36, 43, 52, 54).$$

One notable problem of chord progression generation is the problem of conditional chord voicing. $\newline$
Given two chords, $c_1, c_2$, and a voicing $v_1$, how does one choose a voicing $v_2$ of $c_2$ that "works well" with $c_1, v_1$?

Note this is not easy when the two chords have a different number of notes. First we assume they have the same number of notes.

We will consider the following strategies:

a) Choose the voicing so that the movement from $c_1, v_1$ to $c_2, v_2$ is minimized. Note there could be multiple voicings that satisfy this (maybe).

b) Choose the voicing so that the high register movement is minimized

c) Fix the roots to neighboring octaves, remaining movement is minimized

d) Fix the roots to neighboring octaves, then prioritize high register movement minimzation

e) Some sort of beam search

$c_1=(0, 2, 2, 0, 0, 2, 0), v_1=(3, 4, 3, 0, 0, 4, 0), c2=(5, 2, 1, 0, 3, 0, 0)$, here $c_1, c_2$ are "C #11", and "F b5 #9" respectively.

Now, $N(c_1, v_1)=(36, 43, 52, 54), N(c_2, v_2)=(35, 45, 53, 56)$
The distance from $(c_1, v_1) \text{ to} (c_2, v_2)$ is $\sum\limits_{i=1}^4 |N(c_1, v_1)-N(c_2, v_2)|=1+2+1+2$

Algorithm 1:


In [14]:
#returns a voicing for chord2 that is compatible with voicing1
def conditional_voicing1(voicing1, chord2):
    voicing1 = np.array(voicing1)
    voicing1 = np.sort(voicing1)
    keys1 = keys_of_voicing(voicing1)
    og_keys2 = keys_of_chord(chord2)
    keys2 = np.delete(og_keys2, [0])
    # initializing what we will return
    voicing2 = -1*np.ones(len(keys2), dtype=int)
    # Setting the root notes
    voicing2[0]=keys2[0]
    voicing2[1]=keys2[0]+12

    for i in range(len(voicing1)-1):
        #select the ith highest note in chord 1
        high_note1 = voicing1[-i-1] 
        #initialize an array that will contain the note distances to the high note
        dists = np.ones(len(keys2), dtype=int)
        # fill out the note distances to the high note
        for j in range(len(dists)):
            dist = note_dist(high_note1%12, keys2[j])
            dists[j]=dist


        #index of minimum distance between note and chord
        idx = np.argmin(dists, axis=0)

        print("Step #: ", i+1 ," Note: ", high_note1, ", key: ", high_note1%12)
        print("Distances: ", dists)
        print("Key closest in chord2: ", keys2[idx], " index: ", idx)
        #now we must choose the note for this neighboring key we have identified!
        close_key = keys2[idx]
        corr_note =voicing1[-i-1]-int(math.remainder(high_note1%12-close_key, 12))
        print("Corresponding note in chord2: ", corr_note)
        voicing2[idx]=corr_note
        #keys2[idx] = -1
        
    print(voicing2)

    for i in range(len(voicing2)):
      if voicing2[i]==-1:
        high_note2 = keys2[i]
        dists = np.ones(len(np.delete(keys1, 0)), dtype=int)
        for j in range(len(dists)):
          dist = note_dist(high_note2, np.delete(voicing1, [0, 1])[j])
          dists[j]=dist
        idx = np.argmin(dists, axis=0)
        print("Keys 2: ", keys2)
        print("Voicing 2: ", voicing2)
        print("Voicing 1: ", np.delete(voicing1, [0, 1]))
        print("Keys 1: ", np.delete(keys1, 0))
        print("Distances: ", dists)
        print("Step #: ", i+1 ," Note: ", high_note2,
              ", note closest in chord1: :",
              np.delete(voicing1, [0, 1])[idx], " index: ", idx)
        direction = 1

        close_key = np.delete(keys1, 0)[idx]
        if high_note2 - close_key > note_dist(high_note2, close_key):
            direction = -1
        corr_note = voicing1[idx+2]+direction*note_dist(high_note2, close_key)
        print("Corresponding note in chord1: ", corr_note)
        print("Close note: ", voicing1[idx+2])
        voicing2[i]=corr_note


    voicing2=np.concatenate(([og_keys2[0],og_keys2[0]+12], voicing2))
    voicing2 = voice_correction(voicing2)
    
    print("Voicing2: ", voicing2)
    print("Keys of voicing2: ", keys_of_voicing(voicing2))
    print("Keys of chord2: ", keys_of_chord(chord2))
    return voicing2

voicing1=[10, 22, 37, 41]
chord2 = chord_finder('C#m 13')

print(conditional_voicing1(voicing1, chord2))
print(keys_of_voicing(conditional_voicing1(voicing1, chord2)))
print(keys_of_chord(chord2))
play_voicing(chord2, voice=conditional_voicing1(voicing1, chord2))

Step #:  1  Note:  41 , key:  5
Distances:  [1 3 5]
Key closest in chord2:  4  index:  0
Corresponding note in chord2:  40
Step #:  2  Note:  37 , key:  1
Distances:  [3 5 3]
Key closest in chord2:  4  index:  0
Corresponding note in chord2:  40
Step #:  3  Note:  22 , key:  10
Distances:  [6 2 0]
Key closest in chord2:  10  index:  2
Corresponding note in chord2:  22
[40 16 22]
Voicing2:  [13 25 40 28 34]
Keys of voicing2:  [ 1  4 10]
Keys of chord2:  [ 1  4  8 10]
[13 25 40 28 34]
Step #:  1  Note:  41 , key:  5
Distances:  [1 3 5]
Key closest in chord2:  4  index:  0
Corresponding note in chord2:  40
Step #:  2  Note:  37 , key:  1
Distances:  [3 5 3]
Key closest in chord2:  4  index:  0
Corresponding note in chord2:  40
Step #:  3  Note:  22 , key:  10
Distances:  [6 2 0]
Key closest in chord2:  10  index:  2
Corresponding note in chord2:  22
[40 16 22]
Voicing2:  [13 25 40 28 34]
Keys of voicing2:  [ 1  4 10]
Keys of chord2:  [ 1  4  8 10]
[ 1  4 10]
[ 1  4  8 10]
Step #:  1  Note

In [15]:
def high_notes(voicing):
    voicing_highs = []
    average = 0
    for note in voicing:
        if note >= 28:
            voicing_highs.append(note)
            average += note
    average = average/len(voicing_highs)
    if average == 0:
        average = max(voicing)
    return voicing_highs, average

In [18]:
def all_voicings(chord):
    """
    Returns a collection of reasonable rootless voicings for a chord
    """
    keys = keys_of_chord(chord)
    octaves = [3,4]
    n = len(keys)
    octave_arrays = [octaves]*(n-1)
    octave_arrays.insert(0, [1])
    octave_arrays = tuple(octave_arrays)

    voicings = cartesian_coord(*octave_arrays)
    voicings = [12*voice+keys for voice in voicings]
        
    return voicings

for i, voice in enumerate(all_voicings(chord_finder("A 7 #11"))):
    if i % 5 == 0 or True:
        print(voice)
        play_voicing([2, 2, 2, 2, 0, 2, 2], voice = voice)

[21 37 40 43 39]
[21 37 40 43 51]


KeyboardInterrupt: 

In [16]:
# An alternative conditional voicing function
# Needs to also return voicing with lower variance
def conditional_voicing2(voicing1, chord2):
    """
    Computes the average note in voicing1 except any bass notes
    Considers possible voicings for chord2 and chooses the one
    whose average is closest to voicing1
    
    Inputs: voicing1, the voicing of the first chord
    Inputs: chord2, the chord following the first chord that needs
    a voicing
    
    Outputs: voicing2, a voicing for chord2
    """
    voicing1_highs, average1 = high_notes(voicing1)
    voicing2 = random_initial_root_voicing(chord2)
    voicing2_highs, average2 = high_notes(voicing2)
    # average1 = average1/len(voicing1)
    # average2 = average2/len(voicing2)

    for voice in all_voicings(chord2):
        root=voice[0]%12
        if root < 4:
            root += 12
        
        voice_highs, average = high_notes(voice)
        #average = average/len(voice)
        if abs(average1-average) < abs(average1-average2):
            voicing2 = np.concatenate(([root],voice[1:]))
            average2 = average
    return voice_correction(voicing2)

chord1 = chord_finder('Bbm')
voicing1=np.array([10, 22, 37, 41])
chord2 = chord_finder('C#m 13')
voicing2 = conditional_voicing2(voicing1, chord2)
play_voicing(chord1, voice = voicing1)
play_voicing(chord2, voice = voicing2)

In [32]:
#chord_progression=['Bm', 'Gm', 'Bbm', 'C#m 13']
#chord_progression=['Am', 'Cm', 'Bbm', 'C#m']
#chord_progression=['Bm', 'Gm', 'Em 7', 'Gm']
chord_progression=['G#sus4', 'Am 7', 'C#m 13', 'Cm 13', 'F#sus4', 'G# 7', 'C 7', 'B 7']
n = len(chord_progression)
chord1=chord_finder(chord_progression[0])
voicing1 = random_initial_root_voicing(chord1)
voicing_prev=voicing1
for i in range(4):
    print(voicing1)
    play_voicing(chord1, voice=voicing1)
    for j in range(n-1):
        chord_prev = chord_finder(chord_progression[j])
        chord_cur = chord_finder(chord_progression[j+1])
        voicing_cur = conditional_voicing2(voicing_prev, chord_cur)
        print(voicing_cur)
        play_voicing(chord_cur, voice = voicing_cur)
        voicing_prev = voicing_cur
    voicing1 = conditional_voicing2(voicing_cur, chord1)
    #voicing1 = random_initial_root_voicing(chord1)

[ 8 20 49 39]
[ 9 36 40 43]
[13 40 44 46]
[12 39 43 45]
[ 6 18 47 25]
[ 8 36 51 54]
[12 40 43 46]
[11 23 39 42 45]
[ 8 20 49 27]
[ 9 21 48 40 43]
[13 40 44 46]
[12 39 43 45]
[ 6 18 47 25]
[ 8 36 51 54]
[12 24 40 55 34]
[11 39 42 45]
[ 8 20 49 27]


KeyboardInterrupt: 

In [43]:
def all_voicings(chord):
    """
    Returns a collection of reasonable rootless voicings for a chord
    """
    keys = keys_of_chord(chord)
    octaves = [2,3,4]
    n = len(keys)
    voicings = cartesian_coord(*tuple([octaves]*n))
    for i in range(len(voicings)):
        voicings[i] = 12*voicings[i]+keys
        root = keys[0]
        
    return voicings

for i, voice in enumerate(all_voicings(chord_finder("A 7 #11"))):
    if i % 5 == 0:
        print(voice)
        play_voicing([2, 2, 2, 2, 0, 2, 2], voice = voice)

[33 25 28 31 27]
[33 25 28 43 51]


KeyboardInterrupt: 

In [33]:
for i, row in chords.sample(frac=1).iterrows():
#for i, row in chords.iterrows():
    print(chord_name(row[:7]))
    chord1 = row[:7]
    chord2 = chords.iloc[i+1][:7]
    
    if i == 0:
        voicing1 = random_initial_root_voicing(chord1)
        play_voicing(row[:7])
        voicing2 = conditional_voicing2(voicing1, chord2)
        voicing1 = voicing2
    else:
        voicing2 = conditional_voicing2(voicing1, chord2)
        play_voicing(chord2, voice = voicing2)
        

G#m b5 9 13
G b5 maj7 #9 11 13
Bbm b5 maj7 13
Ebm 11 13
Ebsus4 maj7 b9 #11
C# b5 b9 13
Bbsus2 b5 maj7 b9 11 b13
E b5 maj7
Gm 7 b9 11
G#sus4 b5 13
Dsus2 #5 13
G# 7 b9 #11 13
Eb 9 13
C#sus4 b5 7
Csus4 b9 #11
F#sus2 b5
E 9 11
Dm b5 maj7 b9 11 13
Am maj7 #11 13
Fm b9 #11 13
F b5 7 9
Dm maj7 11 b13
F #5 9
Csus4 7 b9 #11
D 7 #9 13


KeyboardInterrupt: 